# Scraper for Question 2
Scraping necessary data to look at how the winning of different round types impacts the overall win rate of the match.

This data will also be taken from [DreamTeam](https://dreamteam.gg/), this time focusing on the economic type of each round for each team. The various round types include: **pistol rounds** (the first round of each half where teams only have a pistol and kevlar), **eco rounds** (a team doesn't buy anything, choosing to save their money for the next round), **force rounds** (a team doesn't have enough money for a full buy or enough to eco so they buy as best they can), and **full rounds** (a team has the best buy available).

## 2nd Scrape: Economics (DreamTeam)
Firstly, import the necessary libraries for this notebook.

In [1]:
# Import libraries
import pandas as pd
import urllib
import urllib.request
from time import time, sleep
import re
from loguru import logger
import json
from pprint import pprint

Next, functions are written that will be used later for various purposes.

In [2]:
# Function

# Build the URL needed for scraping
def get_url(MATCHCODE):
    url = "https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/" + MATCHCODE + "/economics"
    return url
    
# Uses urllib to retrieve webpage and scrape JSON data
def get_json(url):
    # Retrieve HTML
    req = urllib.request.Request(url, headers={"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3"})
    page = urllib.request.urlopen(req).read()

    # Sleep to prevent overflowing website
    sleep(1)

    # Output of page returned is JSON, no need for BeautifulSoup
    data = json.loads(page)
    return data

# Instead of originally using a for loop and iloc to get Match ID from dfDTLinks 
# Create a function that can then be used with a concat and apply

def scrape_economics(matchID, MATCHCODE):
    
    df = pd.DataFrame(columns=["Match ID", "Round", "Team", "Player", "Side", "Round Type"])
    
    # Get URL
    url = get_url(MATCHCODE)
    
    # Logger message
    logger.info('Scraping new match: %s' % url)
    
    # Get JSON data
    data = get_json(url)

    # Get round count for iterating through first kills
    totalCount = data["total"]

    # Get details needed from each round and append to DataFrame
    for x in range(totalCount):
        roundNum = int(data["items"][x]["round"])
        teamName = data["items"][x]["player"]["team"]
        playerName = data["items"][x]["player"]["nickname"]
        side = data["items"][x]["player"]["side"]
        roundType = data["items"][x]["round_type"]

        # Append
        df = df.append({"Match ID": int(matchID), "Round": roundNum, "Team": teamName, "Player": playerName,
                                   "Side": side, "Round Type": roundType}, ignore_index=True)
        
    return df

The cleaned dataset of the DreamTeam links is loaded into a DataFrame, so that only valid matches are scraped.

In [3]:
# Read in DreamTeam links from CSV file
dfDTLinks = pd.read_csv("DTLINKS_CLEAN.csv", index_col=0)
dfDTLinks

,Event,Dream Team URL,Match ID,MATCHCODE
0,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,82050,5ca2589cbcf70a02726f6a24
1,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,81790,5ca260d3d35cfb023c7b2d94
2,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,81761,5ca262c227b8d8026d731023
3,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74061,5ca273f3d35cfb026f6f35e2
4,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74059,5ca274bbbcf70a0040573a99
5,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74055,5ca274d38da20e02aa25a612
6,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74051,5ca27568c0f32a02720a2203
7,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74049,5ca2760ebcf70a02726f6a2b
8,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74026,5ca276c6d43602024526e6b6
9,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74002,5ca276e2d43602027748b9e4


As with the two previous DreamTeam scrapes, the data needed is kept in JSON format. Using the function written previously, the data is scraped using the corresponding URL. An example is printed below with PrettyPrint. Each item contains information relating to each player in each round of the match. This info includes: player name, team, and round type.

In [4]:
# Get test URL
testURL = get_url(dfDTLinks["MATCHCODE"].iloc[0])

# Get JSON data
data = get_json(testURL)

# Pretty Printer to show example of JSON output
pprint(data)

{'items': [{'eq_saved': 0,
            'eq_start': 0,
            'final_cash': 0,
            'player': {'nickname': 'ESL Spec',
                       'side': 'unassigned',
                       'steam_id': 'STEAM_1:0:475056422',
                       'team': None,
                       'weapon': None},
            'round': 1,
            'round_type': 'pistol',
            'saved_cash': 800,
            'spend_cash': 0,
            'total_cash': 800},
           {'eq_saved': 0,
            'eq_start': 0,
            'final_cash': 3700,
            'player': {'nickname': 'GuardiaN',
                       'side': 'ct',
                       'steam_id': 'STEAM_1:1:6032647',
                       'team': 'FaZe Clan',
                       'weapon': None},
            'round': 1,
            'round_type': 'pistol',
            'saved_cash': 150,
            'spend_cash': 650,
            'total_cash': 800},
           {'eq_saved': 0,
            'eq_start': 0,
            'final_c

: 5450,
            'player': {'nickname': 'rain',
                       'side': 'ct',
                       'steam_id': 'STEAM_1:1:18542739',
                       'team': 'FaZe Clan',
                       'weapon': None},
            'round': 9,
            'round_type': 'full',
            'saved_cash': 3750,
            'spend_cash': 1000,
            'total_cash': 4750},
           {'eq_saved': 0,
            'eq_start': 6550,
            'final_cash': 2150,
            'player': {'nickname': 'NiKo',
                       'side': 'ct',
                       'steam_id': 'STEAM_1:0:40708825',
                       'team': 'FaZe Clan',
                       'weapon': None},
            'round': 9,
            'round_type': 'full',
            'saved_cash': 450,
            'spend_cash': 4000,
            'total_cash': 4450},
           {'eq_saved': 3900,
            'eq_start': 0,
            'final_cash': 7000,
            'player': {'nickname': 'flamie',
                  

},
           {'eq_saved': 0,
            'eq_start': 0,
            'final_cash': 1450,
            'player': {'nickname': 'NiKo',
                       'side': 'terrorist',
                       'steam_id': 'STEAM_1:0:40708825',
                       'team': 'FaZe Clan',
                       'weapon': None},
            'round': 17,
            'round_type': 'full',
            'saved_cash': 50,
            'spend_cash': 3900,
            'total_cash': 3950},
           {'eq_saved': 0,
            'eq_start': 0,
            'final_cash': 3350,
            'player': {'nickname': 'Zeus',
                       'side': 'ct',
                       'steam_id': 'STEAM_1:0:29531372',
                       'team': 'Natus Vincere',
                       'weapon': None},
            'round': 17,
            'round_type': 'force',
            'saved_cash': 100,
            'spend_cash': 1800,
            'total_cash': 1900},
           {'eq_saved': 0,
            'eq_start': 950,
      

As used previoulsy, iterrows is called and gets the MATCHCODE and Match ID for each map. These are passed into the function and a DataFrame of the economics statistics is built.

In [5]:
dfEconomics = pd.DataFrame(columns=["Match ID", "Round", "Team", "Player", "Side", "Round Type"])

for _, row in dfDTLinks.iterrows():
    dfEconomics = dfEconomics.append(scrape_economics(row["Match ID"], row["MATCHCODE"]))
    
dfEconomics

2019-05-10 20:11:48.939 | INFO     | __main__:scrape_economics:32 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5ca2589cbcf70a02726f6a24/economics


2019-05-10 20:11:51.326 | INFO     | __main__:scrape_economics:32 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5ca260d3d35cfb023c7b2d94/economics


2019-05-10 20:11:53.800 | INFO     | __main__:scrape_economics:32 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5ca262c227b8d8026d731023/economics


2019-05-10 20:11:56.196 | INFO     | __main__:scrape_economics:32 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5ca273f3d35cfb026f6f35e2/economics


2019-05-10 20:11:58.517 | INFO     | __main__:scrape_economics:32 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5ca274bbbcf70a0040573a99/economics


...

2019-05-10 20:24:57.645 | INFO     | __main__:scrape_economics:32 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5caa398cdbb10b05f37119b4/economics


2019-05-10 20:24:59.761 | INFO     | __main__:scrape_economics:32 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5caa39bbd43602057e2452d7/economics


2019-05-10 20:25:02.036 | INFO     | __main__:scrape_economics:32 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5caa39bf3cb88705cc46dd23/economics


2019-05-10 20:25:04.143 | INFO     | __main__:scrape_economics:32 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5caa39c3d436020565490336/economics


,Match ID,Round,Team,Player,Side,Round Type
0,82050,1,None,ESL Spec,unassigned,pistol
1,82050,1,FaZe Clan,GuardiaN,ct,pistol
2,82050,1,FaZe Clan,olofmeister,ct,pistol
3,82050,1,FaZe Clan,AdreN,ct,pistol
4,82050,1,Natus Vincere,Edward,terrorist,pistol
5,82050,1,Natus Vincere,Zeus,terrorist,pistol
6,82050,1,Natus Vincere,flamie,terrorist,pistol
7,82050,1,FaZe Clan,rain,ct,pistol
8,82050,1,FaZe Clan,NiKo,ct,pistol
9,82050,1,Natus Vincere,s1mple,terrorist,pistol


## Cleaning the Scraped Data
The collected is then checked for discrepancies.

In some cases, this data includes the observer for the match. In CS:GO, the observer is like the camerman. They control what the viewers see by switching between the first-person views of the players and also third-person views of certain areas of the map.

To remove these entries, a mask is used to drop the entries that do not have a team side.

In [6]:
dfEconomics = dfEconomics[dfEconomics["Side"] != "unassigned"]
dfEconomics

,Match ID,Round,Team,Player,Side,Round Type
1,82050,1,FaZe Clan,GuardiaN,ct,pistol
2,82050,1,FaZe Clan,olofmeister,ct,pistol
3,82050,1,FaZe Clan,AdreN,ct,pistol
4,82050,1,Natus Vincere,Edward,terrorist,pistol
5,82050,1,Natus Vincere,Zeus,terrorist,pistol
6,82050,1,Natus Vincere,flamie,terrorist,pistol
7,82050,1,FaZe Clan,rain,ct,pistol
8,82050,1,FaZe Clan,NiKo,ct,pistol
9,82050,1,Natus Vincere,s1mple,terrorist,pistol
10,82050,1,Natus Vincere,electronic,terrorist,pistol


The DataFrame is then sorted. First, by Match ID, then Round, and finally by Team. This makes it easier to appened the round winner, but also to drop individual players and only keep one row per team per match. This will be shown later.

In [7]:
dfEconomicsSort = dfEconomics.sort_values(by=["Match ID", "Round", "Team"])
dfEconomicsSort

,Match ID,Round,Team,Player,Side,Round Type
0,17523,1,MTS GameGod Wolf,WOLF > R i X,ct,pistol
3,17523,1,MTS GameGod Wolf,WOLF > Ace,ct,pistol
6,17523,1,MTS GameGod Wolf,WOLF > Mithil:D,ct,pistol
8,17523,1,MTS GameGod Wolf,WOLF > RiTz,ct,pistol
9,17523,1,MTS GameGod Wolf,WOLF > astaRRRRR,ct,pistol
1,17523,1,Ninjas in Pyjamas,friberg younited,terrorist,pistol
2,17523,1,Ninjas in Pyjamas,f0rest Komplett.se,terrorist,pistol
4,17523,1,Ninjas in Pyjamas,Fifflaren AKRACING,terrorist,pistol
5,17523,1,Ninjas in Pyjamas,GeT_RiGhT EIZO,terrorist,pistol
7,17523,1,Ninjas in Pyjamas,Xizt SteelSeries,terrorist,pistol


The cleaned round winner dataset is read in. This contains information on both the round winner and the overall match winner.

In [8]:
# Read in DreamTeam links from CSV file
dfRoundWinner = pd.read_csv("ROUNDWINNER_CLEAN.csv", index_col=0)
dfRoundWinner

,Event,Match ID,Round,Winner,Winner Side,Match Winner
0,IEM Katowice 2019,82050,1,FaZe Clan,ct,Natus Vincere
1,IEM Katowice 2019,82050,2,FaZe Clan,ct,Natus Vincere
2,IEM Katowice 2019,82050,3,Natus Vincere,terrorist,Natus Vincere
3,IEM Katowice 2019,82050,4,Natus Vincere,terrorist,Natus Vincere
4,IEM Katowice 2019,82050,5,Natus Vincere,terrorist,Natus Vincere
5,IEM Katowice 2019,82050,6,Natus Vincere,terrorist,Natus Vincere
6,IEM Katowice 2019,82050,7,FaZe Clan,ct,Natus Vincere
7,IEM Katowice 2019,82050,8,FaZe Clan,ct,Natus Vincere
8,IEM Katowice 2019,82050,9,Natus Vincere,terrorist,Natus Vincere
9,IEM Katowice 2019,82050,10,Natus Vincere,terrorist,Natus Vincere


The unnecessary Event column is dropped and the DataFrame is sorted by Match ID and then by Round.

In [9]:
dfRoundWinner = dfRoundWinner.drop(["Event"], axis=1)
dfRoundWinner = dfRoundWinner.sort_values(["Match ID", "Round"])
dfRoundWinner

,Match ID,Round,Winner,Winner Side,Match Winner
8385,17523,1,MTS GameGod Wolf,ct,Ninjas in Pyjamas
8386,17523,2,MTS GameGod Wolf,ct,Ninjas in Pyjamas
8387,17523,3,Ninjas in Pyjamas,terrorist,Ninjas in Pyjamas
8388,17523,4,Ninjas in Pyjamas,terrorist,Ninjas in Pyjamas
8389,17523,5,Ninjas in Pyjamas,terrorist,Ninjas in Pyjamas
8390,17523,6,Ninjas in Pyjamas,terrorist,Ninjas in Pyjamas
8391,17523,7,MTS GameGod Wolf,ct,Ninjas in Pyjamas
8392,17523,8,Ninjas in Pyjamas,terrorist,Ninjas in Pyjamas
8393,17523,9,Ninjas in Pyjamas,terrorist,Ninjas in Pyjamas
8394,17523,10,MTS GameGod Wolf,ct,Ninjas in Pyjamas


The Match ID and Round columns of dfEconomicsSort are converted to integers. This allows dfEconomicsSort and dfRoundWinner to be merged on these two columns as they are now the same object type.

In [10]:
# Convert Match ID columns to int
dfEconomicsSort["Match ID"] = dfEconomicsSort["Match ID"].astype(int)
dfEconomicsSort["Round"] = dfEconomicsSort["Round"].astype(int)

# Merge DataFrames
dfEconomicsPlayer = pd.merge(dfEconomicsSort,dfRoundWinner, on=["Match ID", "Round"])
dfEconomicsPlayer

,Match ID,Round,Team,Player,Side,Round Type,Winner,Winner Side,Match Winner
0,17523,1,MTS GameGod Wolf,WOLF > R i X,ct,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas
1,17523,1,MTS GameGod Wolf,WOLF > Ace,ct,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas
2,17523,1,MTS GameGod Wolf,WOLF > Mithil:D,ct,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas
3,17523,1,MTS GameGod Wolf,WOLF > RiTz,ct,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas
4,17523,1,MTS GameGod Wolf,WOLF > astaRRRRR,ct,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas
5,17523,1,Ninjas in Pyjamas,friberg younited,terrorist,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas
6,17523,1,Ninjas in Pyjamas,f0rest Komplett.se,terrorist,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas
7,17523,1,Ninjas in Pyjamas,Fifflaren AKRACING,terrorist,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas
8,17523,1,Ninjas in Pyjamas,GeT_RiGhT EIZO,terrorist,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas
9,17523,1,Ninjas in Pyjamas,Xizt SteelSeries,terrorist,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas


Two additional columns, "Won Round" and "Won Match", are added to this new DataFrame. These contain boolean values that tell if the team in the row won the round or won the match.

In [11]:
dfEconomicsPlayer["Won Round"] = (dfEconomicsPlayer["Team"]==dfEconomicsPlayer["Winner"])
dfEconomicsPlayer["Won Match"] = (dfEconomicsPlayer["Team"]==dfEconomicsPlayer["Match Winner"])
dfEconomicsPlayer

,Match ID,Round,Team,Player,Side,Round Type,Winner,Winner Side,Match Winner,Won Round,Won Match
0,17523,1,MTS GameGod Wolf,WOLF > R i X,ct,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,True,False
1,17523,1,MTS GameGod Wolf,WOLF > Ace,ct,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,True,False
2,17523,1,MTS GameGod Wolf,WOLF > Mithil:D,ct,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,True,False
3,17523,1,MTS GameGod Wolf,WOLF > RiTz,ct,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,True,False
4,17523,1,MTS GameGod Wolf,WOLF > astaRRRRR,ct,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,True,False
5,17523,1,Ninjas in Pyjamas,friberg younited,terrorist,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,False,True
6,17523,1,Ninjas in Pyjamas,f0rest Komplett.se,terrorist,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,False,True
7,17523,1,Ninjas in Pyjamas,Fifflaren AKRACING,terrorist,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,False,True
8,17523,1,Ninjas in Pyjamas,GeT_RiGhT EIZO,terrorist,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,False,True
9,17523,1,Ninjas in Pyjamas,Xizt SteelSeries,terrorist,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,False,True


The DataFrame is then checked for null values.

In [12]:
dfEconomicsPlayer.isnull().sum()

Match ID        0
Round           0
Team            0
Player          0
Side            0
Round Type      0
Winner          0
Winner Side     0
Match Winner    0
Won Round       0
Won Match       0
dtype: int64

The DataFrame contains no null values, so the next check is to see if every round contain 10 players.

In [13]:
dfPlayerCount = dfEconomicsPlayer.groupby(["Match ID", "Round"])[["Team"]].count()
dfPlayerCount = dfPlayerCount[dfPlayerCount["Team"] != 10]
dfPlayerCount

Team
Match ID Round      
17539    18       11
17550    2        11
         5        11
17551    24       12
17552    1        11
         2        11
17558    15       11
17562    23       11
18700    29       11
18720    1        11
19242    12       11
19244    22       11
         26       12
19247    3        11
22248    1         9
22260    11        9
22265    21        9
25236    16        9
         18       11
25256    2         9
29533    6        14
         7        12
40896    8         9
40967    16        9
40988    23       11
49718    1         9
49728    1        11
49758    27       11
73833    27       11

In [14]:
dfEconomicsPlayer[(dfEconomicsPlayer["Match ID"] == 29533) & (dfEconomicsPlayer["Round"] == 6)]

,Match ID,Round,Team,Player,Side,Round Type,Winner,Winner Side,Match Winner,Won Round,Won Match
42932,29533,6,Luminosity Gaming,FalleN,ct,full,Luminosity Gaming,ct,Luminosity Gaming,True,True
42933,29533,6,Luminosity Gaming,fer,ct,full,Luminosity Gaming,ct,Luminosity Gaming,True,True
42934,29533,6,Luminosity Gaming,TACO,ct,full,Luminosity Gaming,ct,Luminosity Gaming,True,True
42935,29533,6,Luminosity Gaming,coldzera,ct,full,Luminosity Gaming,ct,Luminosity Gaming,True,True
42936,29533,6,mousesports,fnx,terrorist,full,Luminosity Gaming,ct,Luminosity Gaming,False,False
42937,29533,6,mousesports,nex,terrorist,full,Luminosity Gaming,ct,Luminosity Gaming,False,False
42938,29533,6,mousesports,Clarence,ct,full,Luminosity Gaming,ct,Luminosity Gaming,False,False
42939,29533,6,mousesports,Spiidi,terrorist,full,Luminosity Gaming,ct,Luminosity Gaming,False,False
42940,29533,6,mousesports,chrisJ,terrorist,full,Luminosity Gaming,ct,Luminosity Gaming,False,False
42941,29533,6,mousesports,NiKo,terrorist,full,Luminosity Gaming,ct,Luminosity Gaming,False,False


As we can see, there are multiple rounds where there are more and less than 10 players. This can be accounted for PC crashes during the matches where players drop out of the match. As shown in the example above, sometimes bots take the players place if they do not join back quick enough. In CS:GO, bots are AI players in game that replace a player if they leave. They are used in the casual playerbase as it is common for someone to leave midway through a game for various reasons. However, in a professional game they will never play a round, they only appear during pause time when a player has disconnected. In the example above, the bots are: Clarence, Waldo, Frasier, and Allen.

These rounds will be dropped from the DataFrame instead of indivually cleaning the bad matches. This simplfies the process of only keeping two rows per round per match later on.

The multi-level index of the grouped by DataFrame is removed, with the indices becoming columns.

In [15]:
dfBadRounds = dfPlayerCount.reset_index(level=["Match ID", "Round"])
dfBadRounds

,Match ID,Round,Team
0,17539,18,11
1,17550,2,11
2,17550,5,11
3,17551,24,12
4,17552,1,11
5,17552,2,11
6,17558,15,11
7,17562,23,11
8,18700,29,11
9,18720,1,11


Once again using iterrows, the invalid rounds are removed from dfEconomicsPlayer using a mask.

In [16]:
# Iterrows it iterate over dfBadRounds
for _, row in dfBadRounds.iterrows():
    # Remove unwanted rounds with ~
    dfEconomicsPlayer = dfEconomicsPlayer[~((dfEconomicsPlayer["Match ID"] == row["Match ID"]) & (dfEconomicsPlayer["Round"] == row["Round"]))]
    
dfEconomicsPlayer

,Match ID,Round,Team,Player,Side,Round Type,Winner,Winner Side,Match Winner,Won Round,Won Match
0,17523,1,MTS GameGod Wolf,WOLF > R i X,ct,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,True,False
1,17523,1,MTS GameGod Wolf,WOLF > Ace,ct,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,True,False
2,17523,1,MTS GameGod Wolf,WOLF > Mithil:D,ct,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,True,False
3,17523,1,MTS GameGod Wolf,WOLF > RiTz,ct,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,True,False
4,17523,1,MTS GameGod Wolf,WOLF > astaRRRRR,ct,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,True,False
5,17523,1,Ninjas in Pyjamas,friberg younited,terrorist,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,False,True
6,17523,1,Ninjas in Pyjamas,f0rest Komplett.se,terrorist,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,False,True
7,17523,1,Ninjas in Pyjamas,Fifflaren AKRACING,terrorist,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,False,True
8,17523,1,Ninjas in Pyjamas,GeT_RiGhT EIZO,terrorist,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,False,True
9,17523,1,Ninjas in Pyjamas,Xizt SteelSeries,terrorist,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,False,True


The same group by is used to ensure these rounds have been cleaned from the DataFrame.

In [17]:
dfPlayerCount = dfEconomicsPlayer.groupby(["Match ID", "Round"])[["Team"]].count()
dfPlayerCount[dfPlayerCount["Team"] != 10]

,,Team
Match ID,Round,


As we can see, they no longer exist.

Now, by only keeping every fifth row, a new DataFrame can be created that only holds team values, disregarding the players as they are unlikely to be used in the analysis.

If the rounds that had more or less than 10 players were left in the DataFrame, this method would not work.

In [18]:
dfEconomicsTeam = dfEconomicsPlayer.iloc[::5, :]
dfEconomicsTeam = dfEconomicsTeam.drop(["Player"], axis=1)
dfEconomicsTeam

,Match ID,Round,Team,Side,Round Type,Winner,Winner Side,Match Winner,Won Round,Won Match
0,17523,1,MTS GameGod Wolf,ct,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,True,False
5,17523,1,Ninjas in Pyjamas,terrorist,pistol,MTS GameGod Wolf,ct,Ninjas in Pyjamas,False,True
10,17523,2,MTS GameGod Wolf,ct,full,MTS GameGod Wolf,ct,Ninjas in Pyjamas,True,False
15,17523,2,Ninjas in Pyjamas,terrorist,eco,MTS GameGod Wolf,ct,Ninjas in Pyjamas,False,True
20,17523,3,MTS GameGod Wolf,ct,full,Ninjas in Pyjamas,terrorist,Ninjas in Pyjamas,False,False
25,17523,3,Ninjas in Pyjamas,terrorist,full,Ninjas in Pyjamas,terrorist,Ninjas in Pyjamas,True,True
30,17523,4,MTS GameGod Wolf,ct,eco,Ninjas in Pyjamas,terrorist,Ninjas in Pyjamas,False,False
35,17523,4,Ninjas in Pyjamas,terrorist,full,Ninjas in Pyjamas,terrorist,Ninjas in Pyjamas,True,True
40,17523,5,MTS GameGod Wolf,ct,force,Ninjas in Pyjamas,terrorist,Ninjas in Pyjamas,False,False
45,17523,5,Ninjas in Pyjamas,terrorist,full,Ninjas in Pyjamas,terrorist,Ninjas in Pyjamas,True,True


Cloud9 was renamed Cloud9 G2A for a period as part of a sponsorship deal. To combine the records, replace all occurences of Cloud9 G2A with Cloud9.

In [19]:
dfEconomicsTeam["Team"] = dfEconomicsTeam["Team"].replace({"Cloud9 G2A": "Cloud9"})

The cleaned DreamTeam DataFrame and round winner DataFrame are saved locally in CSV files.

In [20]:
dfEconomicsPlayer.to_csv("ECONOMICS_STATS_PLAYERS.csv", header=True, index=True)
dfEconomicsTeam.to_csv("ECONOMICS_STATS_TEAM.csv", header=True, index=False)